In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-10-15 12:41:48--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   189MB/s    in 0.4s    

2021-10-15 12:41:48 (189 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [ ]:
import pandas as pd
df = pd.read_csv('bioactivity_data.csv')

FileNotFoundError: ignored

In [ ]:
df

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL187579,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,7200.0,intermediate
1,CHEMBL188487,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,9400.0,intermediate
2,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500.0,inactive
3,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110.0,inactive
4,CHEMBL187717,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],2000.0,intermediate
...,...,...,...,...
128,CHEMBL2146517,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,10600.0,inactive
129,CHEMBL187460,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,10100.0,inactive
130,CHEMBL363535,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,11500.0,inactive
131,CHEMBL227075,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,10700.0,inactive


In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
df_lipinski = lipinski(df['canonical_smiles'])

In [ ]:
# QED 
mol = Chem.MolFromSmiles('Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21')
Chem.QED.weights_max(mol)

0.7575592206152357

In [ ]:
# finding QED
QED_value = []
for smile in df['canonical_smiles']:
  mol = Chem.MolFromSmiles(smile)
  qed = Chem.QED.weights_max(mol)
  QED_value.append(qed)
  

In [ ]:
d = {'QED':QED_value}
df_qed = pd.DataFrame(d)
df_qed

,QED
0,0.757559
1,0.487042
2,0.485762
3,0.683944
4,0.348717
...,...
128,0.518458
129,0.663042
130,0.588064
131,0.664966


In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)
df_combined

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL187579,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,7200.0,intermediate,281.271,1.89262,0.0,5.0
1,CHEMBL188487,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,9400.0,intermediate,415.589,3.81320,0.0,2.0
2,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500.0,inactive,421.190,2.66050,0.0,4.0
3,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110.0,inactive,293.347,3.63080,0.0,3.0
4,CHEMBL187717,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],2000.0,intermediate,338.344,3.53900,0.0,5.0
...,...,...,...,...,...,...,...,...
128,CHEMBL2146517,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,10600.0,inactive,338.359,3.40102,0.0,5.0
129,CHEMBL187460,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,10100.0,inactive,296.366,3.44330,0.0,3.0
130,CHEMBL363535,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,11500.0,inactive,276.291,4.09564,0.0,3.0
131,CHEMBL227075,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,10700.0,inactive,278.307,3.29102,0.0,3.0


In [ ]:
final_df = pd.concat([df_combined,df_qed], axis=1)
final_df.head()

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,QED
0,CHEMBL187579,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,7200.0,intermediate,281.271,1.89262,0.0,5.0,0.757559
1,CHEMBL188487,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,9400.0,intermediate,415.589,3.81320,0.0,2.0,0.487042
2,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500.0,inactive,421.190,2.66050,0.0,4.0,0.485762
3,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110.0,inactive,293.347,3.63080,0.0,3.0,0.683944
4,CHEMBL187717,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],2000.0,intermediate,338.344,3.53900,0.0,5.0,0.348717


In [ ]:
final_df.to_csv('bioactivity_data_score.csv', index=False)

In [ ]:
# LogP all bioactivity class
final_df.groupby('bioactivity_class')['LogP'].describe()

,count,mean,std,min,25%,50%,75%,max
bioactivity_class,,,,,,,,
active,15.0,3.777084,1.056293,2.4107,2.81550,3.70040,4.314350,6.10120
inactive,104.0,3.969391,1.455105,-0.0554,3.22032,3.99644,4.808425,7.05154
intermediate,14.0,3.594304,1.165892,1.1272,3.31852,3.68656,4.025030,6.30470


In [ ]:
# QED all bioactivity class
final_df.groupby('bioactivity_class')['QED'].describe()

,count,mean,std,min,25%,50%,75%,max
bioactivity_class,,,,,,,,
active,15.0,0.627761,0.156704,0.206778,0.613257,0.674688,0.731628,0.766180
inactive,104.0,0.465817,0.197367,0.035506,0.305394,0.438330,0.650963,0.924827
intermediate,14.0,0.569472,0.157902,0.286913,0.490015,0.553261,0.664966,0.862078


In [ ]:
df_active_class = final_df[final_df['bioactivity_class']=='active']
df_inactive_class = final_df[final_df['bioactivity_class']=='inactive']
df_intermediate = final_df[final_df['bioactivity_class'] == 'intermediate']
df_active_intermediate = final_df[final_df['bioactivity_class'].isin(['active','intermediate'])]

In [ ]:
df_active_class.to_csv('active_class_score.csv')
df_inactive_class.to_csv('inactive_class_score.csv')
df_intermediate.to_csv('intermediate_class_score.csv')
df_active_intermediate.to_csv('active_intermediate_class_score.csv')

###**Calculate QED and LogP of generated drug**

In [ ]:
f = open('no_tuning_result.txt','r')
carnonical_smiles = []
for smile in f : 
  carnonical_smiles.append(smile)

In [ ]:
# QED 
QED_value = []
for smile in carnonical_smiles:
  mol = Chem.MolFromSmiles(smile)
  qed = Chem.QED.weights_max(mol)
  QED_value.append(qed)

In [ ]:
d = {'carnonical_smile':carnonical_smiles,'QED':QED_value}
df_qed_generate = pd.DataFrame(d)
df_qed_generate

,carnonical_smile,QED
0,C=CCC(C)C=C\n,0.500358
1,C=S(C)CCC(CC)CCO\n,0.625270
2,CCC(C)(C)N[SH]=C(C)C(C)N\n,0.419962
3,CCC1=CC=CC(C(=O)CO)=C1\n,0.669503
4,C=S(N=CS)C(C)=CC=C1C=C1\n,0.242727
5,CC1=CC=C(F)C(CN(C)C(C)CO)=C1\n,0.823346
6,CCCN1C2CC(=O)NCC1C(C)C2\n,0.733751
7,CCC(C)C(C#N)C(=O)NC1=CC(C)=CC=C1\n,0.858002
8,OC1C2=CC=CC(=C2)CCCCC2CC21\n,0.697794
9,N=CC1=CC=CC(N=CC2=CC=CN=C2)=C1\n,0.761961


In [ ]:
# logP
df_generate = lipinski(carnonical_smiles)

In [ ]:
final_generate_df = pd.concat([df_qed_generate,df_generate], axis=1)
final_generate_df

,carnonical_smile,QED,MW,LogP,NumHDonors,NumHAcceptors
0,C=CCC(C)C=C\n,0.500358,96.173,2.38460,0.0,0.0
1,C=S(C)CCC(CC)CCO\n,0.625270,176.325,2.11580,1.0,1.0
2,CCC(C)(C)N[SH]=C(C)C(C)N\n,0.419962,190.356,1.68460,3.0,2.0
3,CCC1=CC=CC(C(=O)CO)=C1\n,0.669503,164.204,1.42400,1.0,2.0
4,C=S(N=CS)C(C)=CC=C1C=C1\n,0.242727,197.328,2.96040,1.0,1.0
5,CC1=CC=C(F)C(CN(C)C(C)CO)=C1\n,0.823346,211.280,1.94672,1.0,2.0
6,CCCN1C2CC(=O)NCC1C(C)C2\n,0.733751,196.294,0.99530,1.0,2.0
7,CCC(C)C(C#N)C(=O)NC1=CC(C)=CC=C1\n,0.858002,230.311,3.11940,1.0,2.0
8,OC1C2=CC=CC(=C2)CCCCC2CC21\n,0.697794,202.297,3.08250,1.0,1.0
9,N=CC1=CC=CC(N=CC2=CC=CN=C2)=C1\n,0.761961,209.252,2.82987,1.0,3.0


In [ ]:
final_generate_df.head()

,carnonical_smile,QED,MW,LogP,NumHDonors,NumHAcceptors
0,C=CCC(C)C=C\n,0.500358,96.173,2.3846,0.0,0.0
1,C=S(C)CCC(CC)CCO\n,0.625270,176.325,2.1158,1.0,1.0
2,CCC(C)(C)N[SH]=C(C)C(C)N\n,0.419962,190.356,1.6846,3.0,2.0
3,CCC1=CC=CC(C(=O)CO)=C1\n,0.669503,164.204,1.4240,1.0,2.0
4,C=S(N=CS)C(C)=CC=C1C=C1\n,0.242727,197.328,2.9604,1.0,1.0


In [ ]:
final_generate_df['QED'].describe()

count    32.000000
mean      0.708784
std       0.152630
min       0.242727
25%       0.667417
50%       0.740529
75%       0.820531
max       0.893178
Name: QED, dtype: float64

In [ ]:
final_generate_df['LogP'].describe()

count    32.000000
mean      3.021954
std       1.036792
min       0.995300
25%       2.135275
50%       2.895135
75%       3.785300
max       5.013800
Name: LogP, dtype: float64